In [61]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time
from tabulate import tabulate
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_squared_error, mean_squared_log_error, mean_absolute_error, make_scorer
from sklearn.model_selection import train_test_split

data = pd.read_csv('csv/final_dataset.csv')
print(data.columns)

# Separate features and response variables
X = data.iloc[:, 2:]                                # features
Y = data['temp_measured']                           # response variable: geothermal reservoir measured temperature
print(f'Features of dataset: {X.columns}')
print(f'Number of compenents in features: {X.shape[1]}')
print(Y.head(10))

Index(['well_sample', 'temp_measured', 'pH', 'Na ', 'K', 'Ca', 'Mg', 'Cl',
       'SO4'],
      dtype='object')
Features of dataset: Index(['pH', 'Na ', 'K', 'Ca', 'Mg', 'Cl', 'SO4'], dtype='object')
Number of compenents in features: 7
0    137
1    137
2    137
3    137
4    150
5    116
6    165
7    140
8    115
9    115
Name: temp_measured, dtype: int64


In [62]:
### Desicion tree with GridSearchCV for parameters tuning

from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV

start_time_dt_gs = time.time()

x_train_dt, x_test_dt, y_train_dt, y_test_dt = train_test_split(X, Y, test_size=0.15, random_state=100)

scaler = StandardScaler()
x_train_dt = scaler.fit_transform(x_train_dt)
x_test_dt = scaler.transform(x_test_dt)

# Setting up grid search cross validation for parameters tuning
# param_grid_dt: dictionary of parameters to be tested.
# cv: number of split for cross-validation: 5- fold cross-validation (validación cruzada quíntuple).
# scoring or evaluation metrics: mean squeared error. GridSearchCV maximize the scoring metrics, that's why it's called 'neg_mean_squared_error'.
# The actual calculation is: -1 * mean_squared_error

# Parameters tuning
tree_regressor = DecisionTreeRegressor()
param_grid_dt = {'criterion': ['squared_error'],
            'max_depth': [50, 75, 100],
            'min_samples_split': [5, 10, 25],
            'max_leaf_nodes': [25, 50, 75],
            'min_impurity_decrease': [0, 0.01, 0.1]}

grid_search_cv = GridSearchCV(
    estimator=tree_regressor, 
    param_grid=param_grid_dt, 
    cv=5, 
    verbose=1, 
    scoring='neg_mean_squared_error',
    n_jobs=-1                                         # n_jobs=-1 utilize all the cores avalaible,
    )


grid_search_cv.fit(x_train_dt, y_train_dt)
best_model_dt = grid_search_cv.best_estimator_

print('='*75)
print(f'Desicion tree regressor best parameters: \n{best_model_dt}')
print('='*75)

y_pred_test_dt = best_model_dt.predict(x_test_dt)
y_pred_train_dt = best_model_dt.predict(x_train_dt)

end_time_dt_gs = time.time()
training_time_dt_gs = end_time_dt_gs - start_time_dt_gs

def mean_relative_squared_error(Y_true, Y_pred):
    return np.mean(((Y_true - Y_pred) / Y_true) ** 2)

r2_dt = r2_score(y_test_dt, y_pred_test_dt)
mse_dt = mean_squared_error(y_test_dt, y_pred_test_dt)
mslr_dt = mean_squared_log_error(y_test_dt, y_pred_test_dt)
mae_dt = mean_absolute_error(y_test_dt, y_pred_test_dt)
mrse_dt = mean_relative_squared_error(y_test_dt, y_pred_test_dt)

eval_metrics_dt = {
    'Eval_metrics': ['R2 Score', 'MSE', 'MAE', 'MSLE', 'MRSE', 'Training time'],
    'Desicion_tree_GS': [r2_dt, mse_dt, mae_dt, mslr_dt, mrse_dt, training_time_dt_gs]
}

df_metrics_dt = pd.DataFrame(eval_metrics_dt)
df_metrics_dt.to_csv('metrics/metrics_dt_gs.csv', index=False)

print(tabulate(df_metrics_dt.round(4), headers='keys', tablefmt='pretty', showindex=False))

#joblib.dump(tree_regressor, 'dt_model.joblib')
#print("Decision Tree model saved as 'decision_tree_model.joblib'.")

Fitting 5 folds for each of 81 candidates, totalling 405 fits


Desicion tree regressor best parameters: 
DecisionTreeRegressor(max_depth=100, max_leaf_nodes=75,
                      min_impurity_decrease=0.01, min_samples_split=10)
+---------------+------------------+
| Eval_metrics  | Desicion_tree_GS |
+---------------+------------------+
|   R2 Score    |      0.8944      |
|      MSE      |     786.5621     |
|      MAE      |     19.4172      |
|     MSLE      |      0.0908      |
|     MRSE      |      0.2084      |
| Training time |      2.8688      |
+---------------+------------------+


In [65]:
### Decision tree with RandomizedSearchCV for parameters tuning

from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint, uniform

start_time_dt_rs = time.time()

x_train_dt, x_test_dt, y_train_dt, y_test_dt = train_test_split(X, Y, test_size=0.15, random_state=100)

scaler = StandardScaler()
x_train_dt = scaler.fit_transform(x_train_dt)
x_test_dt = scaler.transform(x_test_dt)

# Parameters tuning with distributions for RandomizedSearchCV
tree_regressor = DecisionTreeRegressor()
param_distributions_dt = {
    'criterion': ['squared_error'],
    'max_depth': randint(50, 100),             
    'min_samples_split': randint(5, 25),        
    'max_leaf_nodes': randint(25, 75),          
    'min_impurity_decrease': uniform(0, 0.1)    
}

random_search_cv = RandomizedSearchCV(
    estimator=tree_regressor, 
    param_distributions=param_distributions_dt, 
    n_iter=50,  # Number of parameter settings sampled
    cv=5, 
    verbose=1, 
    scoring='neg_mean_squared_error',
    random_state=42,
    n_jobs=-1
)

random_search_cv.fit(x_train_dt, y_train_dt)
best_model_dt = random_search_cv.best_estimator_
print('='*75)
print(f'Decision tree regressor best parameters: \n{best_model_dt}')
print('='*75)
y_pred_test_dt = best_model_dt.predict(x_test_dt)
y_pred_train_dt = best_model_dt.predict(x_train_dt)

end_time_dt_rs = time.time()
training_time_dt_rs = end_time_dt_rs - start_time_dt_rs

def mean_relative_squared_error(Y_true, Y_pred):
    return np.mean(((Y_true - Y_pred) / Y_true) ** 2)

r2_dt = r2_score(y_test_dt, y_pred_test_dt)
mse_dt = mean_squared_error(y_test_dt, y_pred_test_dt)
mslr_dt = mean_squared_log_error(y_test_dt, y_pred_test_dt)
mae_dt = mean_absolute_error(y_test_dt, y_pred_test_dt)
mrse_dt = mean_relative_squared_error(y_test_dt, y_pred_test_dt)

eval_metrics_dt = {
    'Eval_metrics': ['R2 Score', 'MSE', 'MAE', 'MSLE', 'MRSE', 'Training time'],
    'Decision_tree_RS': [r2_dt, mse_dt, mae_dt, mslr_dt, mrse_dt, training_time_dt_rs]
}

df_metrics_dt = pd.DataFrame(eval_metrics_dt)
df_metrics_dt.to_csv('metrics/metrics_dt_rs.csv', index=False)

print(tabulate(df_metrics_dt.round(4), headers='keys', tablefmt='pretty', showindex=False))

#joblib.dump(best_model_dt, 'dt_randomized_model.joblib')
#print("Decision Tree model saved as 'dt_randomized_model.joblib'.")

Fitting 5 folds for each of 50 candidates, totalling 250 fits
Decision tree regressor best parameters: 
DecisionTreeRegressor(max_depth=51, max_leaf_nodes=52,
                      min_impurity_decrease=0.005147875124998935,
                      min_samples_split=9)
+---------------+------------------+
| Eval_metrics  | Decision_tree_RS |
+---------------+------------------+
|   R2 Score    |      0.899       |
|      MSE      |     751.9331     |
|      MAE      |     18.8747      |
|     MSLE      |      0.0898      |
|     MRSE      |      0.2059      |
| Training time |      0.1893      |
+---------------+------------------+


In [66]:
### Decision Tree with Optuna hyperparameter optimization

import optuna
from sklearn.model_selection import cross_val_score

start_time_dt_op = time.time()

x_train_dt, x_test_dt, y_train_dt, y_test_dt = train_test_split(X, Y, test_size=0.15, random_state=100)

scaler = StandardScaler()
x_train_dt = scaler.fit_transform(x_train_dt)
x_test_dt = scaler.transform(x_test_dt)

def objective_dt(trial):
    params = {
        'max_depth': trial.suggest_int('max_depth', 50, 100),
        'min_samples_split': trial.suggest_int('min_samples_split', 5, 25),
        'max_leaf_nodes': trial.suggest_int('max_leaf_nodes', 25, 75),
        'min_impurity_decrease': trial.suggest_float('min_impurity_decrease', 0.0, 0.1)
    }
    model = DecisionTreeRegressor(**params, random_state=42)
    score = cross_val_score(model, x_train_dt, y_train_dt, cv=5, scoring='neg_mean_squared_error').mean()
    return -score

optuna_dt = optuna.create_study(direction='minimize')
optuna_dt.optimize(objective_dt, n_trials=40, show_progress_bar=False)

print('='*125)
print(f"Best Decision Tree params: \n{optuna_dt.best_params}")
print('='*125)

best_dt = DecisionTreeRegressor(**optuna_dt.best_params, random_state=42)
best_dt.fit(x_train_dt, y_train_dt)
y_pred_dt = best_dt.predict(x_test_dt)

final_time_dt_op = time.time()
training_time_dt_op = final_time_dt_op - start_time_dt_op

def mean_relative_squared_error(Y_true, Y_pred):
    return np.mean(((Y_true - Y_pred) / Y_true) ** 2)

r2_dt = r2_score(y_test_dt, y_pred_dt)
mse_dt = mean_squared_error(y_test_dt, y_pred_dt)
mae_dt = mean_absolute_error(y_test_dt, y_pred_dt)
msle_dt = mean_squared_log_error(y_test_dt, y_pred_dt)
mrse_dt = mean_relative_squared_error(y_test_dt, y_pred_dt)

dt_eval_metrics = {
    'Eval_metrics': ['R2 Score', 'MSE', 'MAE', 'MSLE', 'MRSE', 'Training time'],
    'Decision_tree_Op': [r2_dt, mse_dt, mae_dt, msle_dt, mrse_dt, training_time_dt_op]
}

dt_df_metrics = pd.DataFrame(dt_eval_metrics)
dt_df_metrics.to_csv('metrics/metrics_dt_op.csv', index=False)

print(tabulate(dt_df_metrics.round(4), headers='keys', tablefmt='pretty', showindex=False))

[I 2025-09-01 10:05:59,284] A new study created in memory with name: no-name-675ab232-53d6-401c-969b-0d8e26aec203
[I 2025-09-01 10:05:59,301] Trial 0 finished with value: 1724.3412773444256 and parameters: {'max_depth': 75, 'min_samples_split': 14, 'max_leaf_nodes': 46, 'min_impurity_decrease': 0.07163843340735598}. Best is trial 0 with value: 1724.3412773444256.
[I 2025-09-01 10:05:59,314] Trial 1 finished with value: 1790.6264050294908 and parameters: {'max_depth': 88, 'min_samples_split': 20, 'max_leaf_nodes': 28, 'min_impurity_decrease': 0.07716611271385704}. Best is trial 0 with value: 1724.3412773444256.
[I 2025-09-01 10:05:59,329] Trial 2 finished with value: 1681.6530064818828 and parameters: {'max_depth': 56, 'min_samples_split': 17, 'max_leaf_nodes': 72, 'min_impurity_decrease': 0.06192589704875291}. Best is trial 2 with value: 1681.6530064818828.
[I 2025-09-01 10:05:59,342] Trial 3 finished with value: 1834.7233076966786 and parameters: {'max_depth': 70, 'min_samples_split':

Best Decision Tree params: 
{'max_depth': 62, 'min_samples_split': 11, 'max_leaf_nodes': 34, 'min_impurity_decrease': 0.04433236717172775}
+---------------+------------------+
| Eval_metrics  | Decision_tree_Op |
+---------------+------------------+
|   R2 Score    |      0.8943      |
|      MSE      |     786.9707     |
|      MAE      |     19.3012      |
|     MSLE      |      0.0961      |
|     MRSE      |      0.2225      |
| Training time |      1.0797      |
+---------------+------------------+


In [71]:
### Random Forest with RandomizedSearchCV for hyperparameter optimization

from sklearn.ensemble import RandomForestRegressor
import time

start_time_rf_rs = time.time()

x_train_rf, x_test_rf, y_train_rf, y_test_rf = train_test_split(X, Y, test_size=0.15, random_state=100)

scaler = StandardScaler()
x_train_rf = scaler.fit_transform(x_train_rf)
x_test_rf = scaler.transform(x_test_rf)

# Define parameter distributions for RandomizedSearchCV
rf_param_distributions = {
    'n_estimators': randint(50, 500),
    'max_depth': randint(5, 50),
    'min_samples_split': randint(2, 20),
    'min_samples_leaf': randint(1, 10),
    'max_features': ['sqrt', 'log2', None],
    'bootstrap': [True, False]
}

# RandomizedSearchCV with Random Forest
rf_regressor = RandomForestRegressor(random_state=42, n_jobs=-1)

random_search_rf = RandomizedSearchCV(
    estimator=rf_regressor,
    param_distributions=rf_param_distributions,
    n_iter=50,  # Number of parameter settings sampled
    cv=5,
    verbose=1,
    scoring='neg_mean_squared_error',
    random_state=42,
    n_jobs=-1
)

random_search_rf.fit(x_train_rf, y_train_rf)
best_rf = random_search_rf.best_estimator_

print("="*135)
print(f"Best Random Forest params: \n{random_search_rf.best_params_}")
print("="*135)
y_pred_rf = best_rf.predict(x_test_rf)

end_time_rf_rs = time.time()
training_time_rf_rs= end_time_rf_rs - start_time_rf_rs

def mean_relative_squared_error(Y_true, Y_pred):
    return np.mean(((Y_true - Y_pred) / Y_true) ** 2)

r2_rf = r2_score(y_test_rf, y_pred_rf)
mse_rf = mean_squared_error(y_test_rf, y_pred_rf)
mae_rf = mean_absolute_error(y_test_rf, y_pred_rf)
msle_rf = mean_squared_log_error(y_test_rf, y_pred_rf)
mrse_rf = mean_relative_squared_error(y_test_rf, y_pred_rf)

rf_eval_metrics = {
    'Eval_metrics': ['R2 Score', 'MSE', 'MAE', 'MSLE', 'MRSE', 'Training time'],
    'Random_forest_RS': [r2_rf, mse_rf, mae_rf, msle_rf, mrse_rf, training_time_rf_rs]
}

rf_df_metrics = pd.DataFrame(rf_eval_metrics)
rf_df_metrics.to_csv('metrics/metrics_rf_rs.csv', index=False)

print(tabulate(rf_df_metrics.round(4), headers='keys', tablefmt='pretty', showindex=False))

Fitting 5 folds for each of 50 candidates, totalling 250 fits


Best Random Forest params: 
{'bootstrap': False, 'max_depth': 34, 'max_features': 'log2', 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 363}
+---------------+------------------+
| Eval_metrics  | Random_forest_RS |
+---------------+------------------+
|   R2 Score    |      0.9558      |
|      MSE      |     328.8683     |
|      MAE      |     14.4024      |
|     MSLE      |      0.0509      |
|     MRSE      |      0.0841      |
| Training time |     14.2937      |
+---------------+------------------+


In [72]:
### Random Forest with Optuna hyperparameter optimization

start_time_rf_op = time.time()

x_train_rf, x_test_rf, y_train_rf, y_test_rf = train_test_split(X, Y, test_size=0.15, random_state=100)

scaler = StandardScaler()
x_train_rf = scaler.fit_transform(x_train_rf)
x_test_rf = scaler.transform(x_test_rf)

def objective_rf(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 500),
        'max_depth': trial.suggest_int('max_depth', 5, 50),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 20),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 10),
        'max_features': trial.suggest_categorical('max_features', ['sqrt', 'log2', None]),
        'bootstrap': trial.suggest_categorical('bootstrap', [True, False])
    }
    model = RandomForestRegressor(**params, random_state=42, n_jobs=-1)
    score = cross_val_score(model, x_train_rf, y_train_rf, cv=5, scoring='neg_mean_squared_error').mean()
    return -score

study_rf = optuna.create_study(direction='minimize')
study_rf.optimize(objective_rf, n_trials=40)

print("Best Random Forest params:", study_rf.best_params)

best_rf = RandomForestRegressor(**study_rf.best_params, random_state=42, n_jobs=-1)
best_rf.fit(x_train_rf, y_train_rf)
y_pred_rf = best_rf.predict(x_test_rf)

end_time_rf_op = time.time()
training_time_rf_op = end_time_rf_op - start_time_rf_op

def mean_relative_squared_error(Y_true, Y_pred):
    return np.mean(((Y_true - Y_pred) / Y_true) ** 2)

r2_rf = r2_score(y_test_rf, y_pred_rf)
mse_rf = mean_squared_error(y_test_rf, y_pred_rf)
mae_rf = mean_absolute_error(y_test_rf, y_pred_rf)
msle_rf = mean_squared_log_error(y_test_rf, y_pred_rf)
mrse_rf = mean_relative_squared_error(y_test_rf, y_pred_rf)

rf_eval_metrics = {
    'Eval_metrics': ['R2 Score', 'MSE', 'MAE', 'MSLE', 'MRSE', 'Training time'],
    'Random_forest_Op': [r2_rf, mse_rf, mae_rf, msle_rf, mrse_rf, training_time_rf_op]
}

rf_df_metrics = pd.DataFrame(rf_eval_metrics)
rf_df_metrics.to_csv('metrics/metrics_rf_op.csv', index=False)

print(tabulate(rf_df_metrics.round(4), headers='keys', tablefmt='pretty', showindex=False))

[I 2025-09-01 11:01:00,917] A new study created in memory with name: no-name-0c5f7ab5-00a0-4f3c-af09-4795fb0d708d


[I 2025-09-01 11:01:03,542] Trial 0 finished with value: 1852.6939176950268 and parameters: {'n_estimators': 442, 'max_depth': 14, 'min_samples_split': 3, 'min_samples_leaf': 10, 'max_features': 'sqrt', 'bootstrap': True}. Best is trial 0 with value: 1852.6939176950268.
[I 2025-09-01 11:01:03,971] Trial 1 finished with value: 1891.858699275574 and parameters: {'n_estimators': 83, 'max_depth': 22, 'min_samples_split': 3, 'min_samples_leaf': 3, 'max_features': None, 'bootstrap': False}. Best is trial 0 with value: 1852.6939176950268.
[I 2025-09-01 11:01:06,195] Trial 2 finished with value: 1553.4437953514027 and parameters: {'n_estimators': 387, 'max_depth': 27, 'min_samples_split': 20, 'min_samples_leaf': 4, 'max_features': 'log2', 'bootstrap': True}. Best is trial 2 with value: 1553.4437953514027.
[I 2025-09-01 11:01:07,999] Trial 3 finished with value: 1841.0486087514037 and parameters: {'n_estimators': 469, 'max_depth': 9, 'min_samples_split': 15, 'min_samples_leaf': 6, 'max_features

Best Random Forest params: {'n_estimators': 342, 'max_depth': 40, 'min_samples_split': 4, 'min_samples_leaf': 2, 'max_features': 'log2', 'bootstrap': False}
+---------------+------------------+
| Eval_metrics  | Random_forest_Op |
+---------------+------------------+
|   R2 Score    |      0.9562      |
|      MSE      |     325.8803     |
|      MAE      |     14.3941      |
|     MSLE      |      0.0504      |
|     MRSE      |      0.0829      |
| Training time |     58.8496      |
+---------------+------------------+


In [45]:
### XGBoost implementation with RandomizedSearchCV for faster hyperparametr tuning
'''
Unlike GridSearchCV, which tries all possible combiationn of hyperparameters,
RandomizedSearchCV samples a fixed number of hyperparameters combinations from
the specified space. This is useful when you have a large hyprparameteres space,
as it saves time by exploring only a subset of all possible combinations.
'''

import xgboost as xgb

start_time_xg_rs = time.time()

x_train_xg, x_test_xg, y_train_log_xg, y_test_log_xg = train_test_split(X, np.log(Y), test_size = 0.15, random_state = 100)

scaler = StandardScaler()
x_train_xg = scaler.fit_transform(x_train_xg)
x_test_xg = scaler.transform(x_test_xg)

xgb_regressor = xgb.XGBRegressor() # (tree_method='gpu_hist', predictor='gpu_predictor', use_label_encoder=False)   # Use GPU for training

# Convert data into DMatrix (XGBoost optimized data structure)
dtrain = xgb.DMatrix(x_train_xg, label=y_train_log_xg)
dtest = xgb.DMatrix(x_test_xg, label=y_test_log_xg)

# Define the space of hyperparameters
xgb_param_distributions = {
    'colsample_bytree': uniform(0.1, 0.9),
    'learning_rate': uniform(0.00001, 0.99999),
    'max_depth': randint(1, 100),
    'n_estimators': randint(5, 10000),
    'subsample': uniform(0.1, 0.9),
    'reg_alpha': uniform(1e-8, 0.99),
    'reg_lambda': uniform(1e-8, 0.99)
}



# Define scoring functions
scorers = {
    'r2': make_scorer(r2_score),
    'neg_mse': make_scorer(mean_squared_error, greater_is_better=False)
}

# RandomizedSearchCV with cross-validation to optimize hyperparameter tuning
random_search_xgb = RandomizedSearchCV(
    estimator=xgb_regressor,
    param_distributions=xgb_param_distributions,
    scoring=scorers,
    refit='r2',         # Refit the model using R2 after searching
    cv=5,              # 10-fold cross-validation
    n_iter=25,         # Number of random combinations to try
    verbose=1,
    random_state=42,
    n_jobs=-1           # -1 to use all processors avalaible
)

random_search_xgb.fit(x_train_xg, y_train_log_xg)

xgb_best_params = random_search_xgb.best_params_

print("="*200)
print(f'Best XGBoost hyperparameters: \n{xgb_best_params}')
print("="*200)

#params_filename = 'xgbRS_best_params.json'
#with open(params_filename, 'w') as params_file:
#    json.dump(xgb_best_params, params_file)
#print(f'Best XGBoost hyperparameters saved to: \n{params_filename}')

xgb_regressor = xgb.XGBRegressor(**xgb_best_params, 
                                 eval_metrics='rmse', 
                                 early_stopping_rounds=1500) #, tree_method='gpu_hist', predictor='gpu_predictor',)  ## Use GPU for training

xgb_model = xgb_regressor.fit(x_train_xg, 
                              y_train_log_xg, 
                              verbose=False, 
                              eval_set = [(x_train_xg, y_train_log_xg), (x_test_xg, y_test_log_xg)])

end_time_xg_rs = time.time()
training_time_xgb_rs = end_time_xg_rs - start_time_xg_rs

# Save the trained XGBoost model to a file
# model_filename = 'xgbRS_model.json'
# xgb_model.save_model(model_filename)
# print(f'XGBoost model saved to {model_filename}')

y_pred_log_test_xg = xgb_model.predict(x_test_xg)
y_pred_log_train_xg = xgb_model.predict(x_train_xg)

y_pred_test_xg = np.exp(y_pred_log_test_xg)
y_pred_train_xg = np.exp(y_pred_log_train_xg)
y_train_xg = np.exp(y_train_log_xg)
y_test_xg = np.exp(y_test_log_xg)

def mean_relative_squared_error(Y_true, Y_pred):
    return np.mean(((Y_true - Y_pred) / Y_true) ** 2)

r2_xgb= r2_score(y_test_xg, y_pred_test_xg)
mse_xgb = mean_squared_error(y_test_xg, y_pred_test_xg)
mae_xgb = mean_absolute_error(y_test_xg, y_pred_test_xg)
mslr_xgb = mean_squared_log_error(y_test_xg, y_pred_test_xg)
mrse_xgb = mean_relative_squared_error(y_test_xg, y_pred_test_xg)

xgb_eval_metrics = {
    'Eval_metrics': ['R2 Score', 'MSE', 'MAE', 'MSLE', 'MRSE', 'Training time'],
    'XGBoost_RS': [r2_xgb, mse_xgb, mae_xgb, mslr_xgb, mrse_xgb, training_time_xgb_rs]
}

xgb_df_metrics = pd.DataFrame(xgb_eval_metrics)
xgb_df_metrics.to_csv('metrics/metrics_xgb_rs.csv', index=False)

print(tabulate(xgb_df_metrics.round(4), headers='keys', tablefmt='pretty', showindex=False))

Fitting 5 folds for each of 25 candidates, totalling 125 fits
Best XGBoost hyperparameters: 
{'colsample_bytree': 0.4003377500251196, 'learning_rate': 0.1428753892537616, 'max_depth': 3, 'n_estimators': 1690, 'reg_alpha': 0.05584747323682925, 'reg_lambda': 0.7147787945441565, 'subsample': 0.9446974381141752}


/home/chris/venv/cdd/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [19:34:56] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "eval_metrics" } are not used.

  warnings.warn(smsg, UserWarning)


+---------------+------------+
| Eval_metrics  | XGBoost_RS |
+---------------+------------+
|   R2 Score    |   0.9434   |
|      MSE      |  421.583   |
|      MAE      |  15.7383   |
|     MSLE      |   0.0359   |
|     MRSE      |   0.0455   |
| Training time |   8.4571   |
+---------------+------------+


In [60]:
### Enhanced XGBoost implementation with improved hyperparameter tuning

import xgboost as xgb
from scipy.stats import uniform, randint

start_time_xg_rs = time.time()

x_train_xg, x_test_xg, y_train_log_xg, y_test_log_xg = train_test_split(X, np.log(Y), test_size=0.15, random_state=100)

scaler = StandardScaler()
x_train_xg = scaler.fit_transform(x_train_xg)
x_test_xg = scaler.transform(x_test_xg)

# Enhanced hyperparameter space with better ranges
xgb_param_distributions = {
    'colsample_bytree': uniform(0.6, 0.4),          # Focus on higher values (0.6-1.0)
    'learning_rate': uniform(0.01, 0.2),            # More conservative learning rates
    'max_depth': randint(4, 12),                    # Deeper trees for complex patterns
    'n_estimators': randint(1000, 5000),            # More estimators for better fit
    'subsample': uniform(0.8, 0.2),                 # Higher subsample ratios
    'reg_alpha': uniform(0, 0.1),                   # L1 regularization
    'reg_lambda': uniform(0.1, 0.9),                # L2 regularization
    'min_child_weight': randint(1, 10),             # Control overfitting
    'gamma': uniform(0, 0.5),                       # Minimum loss reduction
    'colsample_bylevel': uniform(0.7, 0.3),         # Column sampling by level
    'colsample_bynode': uniform(0.7, 0.3)           # Column sampling by node
}

# Multiple scoring metrics for better model selection
scorers = {
    'r2': make_scorer(r2_score),
    #'neg_mse': make_scorer(mean_squared_error, greater_is_better=False),
    #'neg_mae': make_scorer(mean_absolute_error, greater_is_better=False)
}

# Increased iterations and better cross-validation
random_search_xgb = RandomizedSearchCV(
    estimator=xgb.XGBRegressor(
        random_state=42,
        n_jobs=1,  # XGBoost handles parallelization internally
        verbosity=0
    ),
    param_distributions=xgb_param_distributions,
    scoring=scorers,
    refit='r2',
    cv=10,              # Increased CV folds for better validation
    n_iter=50,          # More iterations for better hyperparameter search
    verbose=1,
    random_state=42,
    n_jobs=-1
)

random_search_xgb.fit(x_train_xg, y_train_log_xg)
xgb_best_params = random_search_xgb.best_params_

print("="*200)
print(f'Best XGBoost hyperparameters: \n{xgb_best_params}')
print("="*200)

# Train final model with optimized parameters and early stopping
xgb_regressor = xgb.XGBRegressor(
    **xgb_best_params,
    random_state=42,
    early_stopping_rounds=100,  # More aggressive early stopping
    eval_metric='rmse',  # Fixed parameter name
    n_jobs=-1
)

# Fit with validation monitoring
xgb_model = xgb_regressor.fit(
    x_train_xg, 
    y_train_log_xg,
    eval_set=[(x_train_xg, y_train_log_xg), (x_test_xg, y_test_log_xg)],
    verbose=False
)

end_time_xg_rs = time.time()
training_time_xgb_rs = end_time_xg_rs - start_time_xg_rs


y_pred_log_test_xg = xgb_model.predict(x_test_xg)
y_pred_log_train_xg = xgb_model.predict(x_train_xg)

y_pred_test_xg = np.exp(y_pred_log_test_xg)
y_pred_train_xg = np.exp(y_pred_log_train_xg)
y_train_xg = np.exp(y_train_log_xg)
y_test_xg = np.exp(y_test_log_xg)

def mean_relative_squared_error(Y_true, Y_pred):
    return np.mean(((Y_true - Y_pred) / Y_true) ** 2)

r2_xgb = r2_score(y_test_xg, y_pred_test_xg)
mse_xgb = mean_squared_error(y_test_xg, y_pred_test_xg)
mae_xgb = mean_absolute_error(y_test_xg, y_pred_test_xg)
mslr_xgb = mean_squared_log_error(y_test_xg, y_pred_test_xg)
mrse_xgb = mean_relative_squared_error(y_test_xg, y_pred_test_xg)

xgb_eval_metrics = {
    'Eval_metrics': ['R2 Score', 'MSE', 'MAE', 'MSLE', 'MRSE', 'Training time'],
    'XGBoost_Enhanced': [r2_xgb, mse_xgb, mae_xgb, mslr_xgb, mrse_xgb, training_time_xgb_rs]
}

xgb_df_metrics = pd.DataFrame(xgb_eval_metrics)
xgb_df_metrics.to_csv('metrics/metrics_xgb_enhanced.csv', index=False)

print(tabulate(xgb_df_metrics.round(4), headers='keys', tablefmt='pretty', showindex=False))

Fitting 10 folds for each of 50 candidates, totalling 500 fits
Best XGBoost hyperparameters: 
{'colsample_bylevel': 0.9876107222223538, 'colsample_bynode': 0.7173591693273121, 'colsample_bytree': 0.7578087194432519, 'gamma': 0.05337803296794219, 'learning_rate': 0.07713201295939263, 'max_depth': 5, 'min_child_weight': 4, 'n_estimators': 1089, 'reg_alpha': 0.03882683839898722, 'reg_lambda': 0.3064552711344547, 'subsample': 0.8531875849928479}
+---------------+------------------+
| Eval_metrics  | XGBoost_Enhanced |
+---------------+------------------+
|   R2 Score    |      0.9426      |
|      MSE      |     427.2001     |
|      MAE      |     16.7496      |
|     MSLE      |      0.0378      |
|     MRSE      |      0.0478      |
| Training time |     17.8854      |
+---------------+------------------+


In [82]:
### XGBoost with Optuna hyperparameter optimization

start_time_xg_op = time.time()

x_train_xg, x_test_xg, y_train_log_xg, y_test_log_xg = train_test_split(X, np.log(Y), test_size=0.15, random_state=100)

scaler = StandardScaler()
x_train_xg = scaler.fit_transform(x_train_xg)
x_test_xg = scaler.transform(x_test_xg)

def objective_xgb(trial):
    params = {
        'max_depth': trial.suggest_int('max_depth', 1, 100),
        'learning_rate': trial.suggest_float('learning_rate', 1e-5, 1.0, log=True),
        'n_estimators': trial.suggest_int('n_estimators', 1, 10000),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 20),
        'gamma': trial.suggest_float('gamma', 1e-8, 1.0, log=True),
        'subsample': trial.suggest_float('subsample', 0.1, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.1, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-8, 1.0, log=True),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-8, 1.0, log=True)
    }
    
    model = xgb.XGBRegressor(**params, random_state=42)
    score = cross_val_score(model, x_train_xg, y_train_log_xg, 
                          cv=10, scoring='r2').mean()
    return -score

study_xgb = optuna.create_study(direction='minimize')
study_xgb.optimize(objective_xgb, n_trials=10)

print("="*100)
print(f"Best XGBoost parameters: \n{study_xgb.best_params}")
print("="*100)

#params_filename = 'xgb_optuna_best_params.json'
#with open(params_filename, 'w') as params_file:
#    json.dump(study_xgb.best_params, params_file)
#print(f'Best parameters saved to: {params_filename}')

best_xgb = xgb.XGBRegressor(**study_xgb.best_params, 
                            random_state=42,  
                            eval_metrics='rmse', 
                            early_stopping_rounds=1500) #, tree_method='gpu_hist', predictor='gpu_predictor',)  ## Use GPU for training)

best_xgb.fit(x_train_xg, 
             y_train_log_xg,
             verbose = True,
             eval_set = [(x_train_xg, y_train_log_xg), (x_test_xg, y_test_log_xg)]
             )

end_time_xg_op = time.time()
training_time_xg_op = end_time_xg_op - start_time_xg_op

#model_filename = 'xgb_optuna_model.json'
#best_xgb.save_model(model_filename)
#print(f'Model saved to: {model_filename}')

y_pred_log_test_xg = best_xgb.predict(x_test_xg)
y_pred_log_train_xg = best_xgb.predict(x_train_xg)

y_pred_test_xg = np.exp(y_pred_log_test_xg)
y_pred_train_xg = np.exp(y_pred_log_train_xg)
y_train_xg = np.exp(y_train_log_xg)
y_test_xg = np.exp(y_test_log_xg)

r2_xgb = r2_score(y_test_xg, y_pred_test_xg)
mse_xgb = mean_squared_error(y_test_xg, y_pred_test_xg)
mae_xgb = mean_absolute_error(y_test_xg, y_pred_test_xg)
msle_xgb = mean_squared_log_error(y_test_xg, y_pred_test_xg)
mrse_xgb = mean_relative_squared_error(y_test_xg, y_pred_test_xg)

xgb_eval_metrics = {
    'Eval_metrics': ['R2 Score', 'MSE', 'MAE', 'MSLE', 'MRSE', 'Training time'],
    'XGBoost_Op': [r2_xgb, mse_xgb, mae_xgb, msle_xgb, mrse_xgb, training_time_xg_op]
}

xgb_df_metrics = pd.DataFrame(xgb_eval_metrics)
xgb_df_metrics.to_csv('metrics/metrics_xgb_op.csv', index=False)

print(tabulate(xgb_df_metrics.round(4), headers='keys', tablefmt='pretty', showindex=False))

#Plot optimization history
#optuna.visualization.plot_optimization_history(study_xgb)
#plt.show()

[I 2025-09-01 18:02:16,128] A new study created in memory with name: no-name-a1371493-072b-44ed-a528-5cdca656d07e
[I 2025-09-01 18:02:26,276] Trial 0 finished with value: 0.047513697837644864 and parameters: {'max_depth': 70, 'learning_rate': 1.4057274587120775e-05, 'n_estimators': 4921, 'min_child_weight': 11, 'gamma': 0.00022975948849377308, 'subsample': 0.21001445258538975, 'colsample_bytree': 0.28989971424623934, 'reg_alpha': 0.00028819162118379584, 'reg_lambda': 0.00043449075177054116}. Best is trial 0 with value: 0.047513697837644864.
[I 2025-09-01 18:02:48,320] Trial 1 finished with value: -0.7940736146029873 and parameters: {'max_depth': 26, 'learning_rate': 0.004837209887240291, 'n_estimators': 2362, 'min_child_weight': 4, 'gamma': 1.1181269536189042e-08, 'subsample': 0.9914258031383143, 'colsample_bytree': 0.3842331272834272, 'reg_alpha': 0.019439536468303428, 'reg_lambda': 1.4238471731057919e-08}. Best is trial 1 with value: -0.7940736146029873.
[I 2025-09-01 18:03:01,644] T

Best XGBoost parameters: 
{'max_depth': 44, 'learning_rate': 0.005300233254488905, 'n_estimators': 6875, 'min_child_weight': 1, 'gamma': 0.07998120566230651, 'subsample': 0.8323019428349318, 'colsample_bytree': 0.905111591519332, 'reg_alpha': 0.2993213293391337, 'reg_lambda': 0.2116567499019213}
[0]	validation_0-rmse:0.61426	validation_1-rmse:0.66207
[1]	validation_0-rmse:0.61166	validation_1-rmse:0.65962
[2]	validation_0-rmse:0.60900	validation_1-rmse:0.65776
[3]	validation_0-rmse:0.60636	validation_1-rmse:0.65528
[4]	validation_0-rmse:0.60367	validation_1-rmse:0.65253
[5]	validation_0-rmse:0.60096	validation_1-rmse:0.64987
[6]	validation_0-rmse:0.59823	validation_1-rmse:0.64738
[7]	validation_0-rmse:0.59566	validation_1-rmse:0.64467
[8]	validation_0-rmse:0.59324	validation_1-rmse:0.64178
[9]	validation_0-rmse:0.59073	validation_1-rmse:0.63959
[10]	validation_0-rmse:0.58820	validation_1-rmse:0.63721
[11]	validation_0-rmse:0.58582	validation_1-rmse:0.63464
[12]	validation_0-rmse:0.5834

/home/chris/venv/cdd/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [18:06:29] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "eval_metrics" } are not used.

  warnings.warn(smsg, UserWarning)


[64]	validation_0-rmse:0.46776	validation_1-rmse:0.52645
[65]	validation_0-rmse:0.46591	validation_1-rmse:0.52506
[66]	validation_0-rmse:0.46405	validation_1-rmse:0.52366
[67]	validation_0-rmse:0.46203	validation_1-rmse:0.52185
[68]	validation_0-rmse:0.46023	validation_1-rmse:0.52014
[69]	validation_0-rmse:0.45827	validation_1-rmse:0.51843
[70]	validation_0-rmse:0.45627	validation_1-rmse:0.51665
[71]	validation_0-rmse:0.45443	validation_1-rmse:0.51463
[72]	validation_0-rmse:0.45249	validation_1-rmse:0.51281
[73]	validation_0-rmse:0.45070	validation_1-rmse:0.51109
[74]	validation_0-rmse:0.44894	validation_1-rmse:0.50955
[75]	validation_0-rmse:0.44730	validation_1-rmse:0.50811
[76]	validation_0-rmse:0.44532	validation_1-rmse:0.50640
[77]	validation_0-rmse:0.44360	validation_1-rmse:0.50507
[78]	validation_0-rmse:0.44166	validation_1-rmse:0.50366
[79]	validation_0-rmse:0.43976	validation_1-rmse:0.50201
[80]	validation_0-rmse:0.43799	validation_1-rmse:0.50032
[81]	validation_0-rmse:0.43607	

The 'scoring' parameter of cross_val_score must be a str among {'roc_auc_ovo', 'top_k_accuracy', 'f1_samples', 'f1', 'matthews_corrcoef', 'max_error', 'neg_mean_squared_log_error', 'recall_micro', 'roc_auc_ovr_weighted', 'positive_likelihood_ratio', 'explained_variance', 'neg_negative_likelihood_ratio', 'recall_samples', 'jaccard_weighted', 'normalized_mutual_info_score', 'neg_mean_gamma_deviance', 'neg_median_absolute_error', 'roc_auc_ovr', 'neg_mean_squared_error', 'neg_brier_score', 'rand_score', 'balanced_accuracy', 'jaccard_micro', 'accuracy', 'jaccard_samples', 'adjusted_mutual_info_score', 'neg_mean_absolute_error', 'roc_auc_ovo_weighted', 'average_precision', 'r2', 'f1_macro', 'precision', 'precision_micro', 'precision_macro', 'recall', 'recall_macro', 'homogeneity_score', 'adjusted_rand_score', 'neg_log_loss', 'roc_auc', 'neg_mean_absolute_percentage_error', 'v_measure_score', 'recall_weighted', 'fowlkes_mallows_score', 'mutual_info_score', 'neg_root_mean_squared_error', 'completeness_score', 'neg_root_mean_squared_log_error', 'precision_weighted', 'd2_absolute_error_score', 'f1_weighted', 'neg_mean_poisson_deviance', 'jaccard', 'jaccard_macro', 'precision_samples', 'f1_micro'}, a callable or None. Got {'r2': make_scorer(r2_score, response_method='predict'), 'neg_mse': make_scorer(mean_squared_error, greater_is_better=False, response_method='predict'), 'neg_mae': make_scorer(mean_absolute_error, greater_is_better=False, response_method='predict')} instead.

In [81]:
### TabPFN for Temperature Prediction

from tabpfn import TabPFNRegressor

start_time_tabpfn = time.time()

x_train_tab, x_test_tab, y_train_log_tab, y_test_log_tab = train_test_split(X, np.log(Y), test_size=0.15, random_state=100)

scaler = StandardScaler()
x_train_tab = scaler.fit_transform(x_train_tab)
x_test_tab = scaler.transform(x_test_tab)

tabpfn_regressor = TabPFNRegressor(
    device='cpu',  # Use 'cuda' if you have a GPU
)

tabpfn_regressor.fit(x_train_tab, y_train_log_tab)

y_pred_log_test = tabpfn_regressor.predict(x_test_tab)
y_pred_log_train = tabpfn_regressor.predict(x_train_tab)

y_pred_test = np.exp(y_pred_log_test)
y_pred_train = np.exp(y_pred_log_train)
y_train = np.exp(y_train_log_tab)
y_test = np.exp(y_test_log_tab)

end_time_tabpfn = time.time()
training_time_tabpfn = end_time_tabpfn - start_time_tabpfn

def mean_relative_squared_error(Y_true, Y_pred):
    return np.mean(((Y_true - Y_pred) / Y_true) ** 2)

r2_tabpfn = r2_score(y_test, y_pred_test)
mse_tabpfn = mean_squared_error(y_test, y_pred_test)
mae_tabpfn = mean_absolute_error(y_test, y_pred_test)
msle_tabpfn = mean_squared_log_error(y_test, y_pred_test)
mrse_tabpfn = mean_relative_squared_error(y_test, y_pred_test)

tabpfn_eval_metrics = {
    'Eval_metrics': ['R2 Score', 'MSE', 'MAE', 'MSLE', 'MRSE', 'Training time'],
    'TabPFN Model': [r2_tabpfn, mse_tabpfn, mae_tabpfn, msle_tabpfn, mrse_tabpfn, training_time_tabpfn]
}

tabpfn_df_metrics = pd.DataFrame(tabpfn_eval_metrics)
tabpfn_df_metrics.to_csv('metrics/metrics_tabpfn.csv', index=False)

print(tabulate(tabpfn_df_metrics.round(4), headers='keys', tablefmt='pretty', showindex=False))


/home/chris/venv/cdd/lib/python3.12/site-packages/torch/utils/_pytree.py:185: FutureWarning: optree is installed but the version is too old to support PyTorch Dynamo in C++ pytree. C++ pytree support is disabled. Please consider upgrading optree using `python3 -m pip install --upgrade 'optree>=0.13.0'`.
  warnings.warn(


+---------------+--------------+
| Eval_metrics  | TabPFN Model |
+---------------+--------------+
|   R2 Score    |    0.9599    |
|      MSE      |   298.9288   |
|      MAE      |    12.156    |
|     MSLE      |    0.0178    |
|     MRSE      |    0.0163    |
| Training time |    9.4587    |
+---------------+--------------+
